In [2]:

#with caloires
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from sklearn.preprocessing import LabelEncoder
import torch
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.ops import nms
from PIL import Image, ImageDraw, ImageFont
import torchvision.transforms as T

# model saving
model_path = r'C:\Users\tw93\OneDrive\Desktop\masters_project\trained_model.keras'

# Load the trained classification model
classification_model = load_model(model_path)

# Load a pre-trained object detection model
detection_model = fasterrcnn_resnet50_fpn(pretrained=True)
detection_model.eval()

# Image dimensions for classification
img_height = 128
img_width = 128
channels = 1  

#  preprocess the image for classification
def preprocess_image(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (img_width, img_height))
    img = img.astype('float32') / 255.0
    img = img.reshape(1, img_height, img_width, channels)  
    return img

#  decode the predicted label
def decode_label(label, label_encoder):
    return label_encoder.inverse_transform([label])[0]


classes = [
    'apple', 'banana', 'bhatura', 'brownie', 'burger', 'cake',
    'chapati', 'chole', 'dhokla', 'fries', 'grapes', 'gulabjamun',
    'kababs', 'kadhipakoda', 'mango', 'meduvada', 'modak', 'noodles',
    'pizza', 'pomogranate', 'rice', 'roll', 'samosa', 'strawberry',
    'vadapav', 'watermelon'
]


calories = {
    'apple': 52, 'banana': 96, 'bhatura': 277, 'brownie': 466, 'burger': 354,
    'cake': 257, 'chapati': 104, 'chole': 164, 'dhokla': 152, 'fries': 312,
    'grapes': 69, 'gulabjamun': 175, 'kababs': 250, 'kadhipakoda': 150,
    'mango': 60, 'meduvada': 334, 'modak': 200, 'noodles': 138, 'pizza': 285,
    'pomogranate': 83, 'rice': 130, 'roll': 300, 'samosa': 262, 'strawberry': 32,
    'vadapav': 290, 'watermelon': 30
}

# fiiting lable encoder in the classes
label_encoder = LabelEncoder()
label_encoder.fit(classes)

new_image_path = input("Enter the path to the image: ")
new_image = preprocess_image(new_image_path)

prediction = classification_model.predict(new_image)
predicted_class = np.argmax(prediction, axis=1)[0]

# decoding predected label according to the class
predicted_label = decode_label(predicted_class, label_encoder)
print(f"Predicted class index: {predicted_class}, Predicted label: {predicted_label}")

#  detect objects in an image
def detect_objects(image_path, model, threshold=0.5):
    image = Image.open(image_path).convert("RGB")
    transform = T.Compose([T.ToTensor()])
    image_tensor = transform(image)
    with torch.no_grad():
        predictions = model([image_tensor])
        
    boxes = predictions[0]['boxes'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # reducing the not required bounding boxes
    keep = nms(torch.tensor(boxes), torch.tensor(scores), iou_threshold=0.3)
    boxes = boxes[keep].astype(int)
    scores = scores[keep]

    valid_boxes = []
    valid_labels = []

    # Filter boxes based on threshold
    for idx, box in enumerate(boxes):
        if scores[idx] >= threshold:
            valid_boxes.append(box)
            valid_labels.append(predicted_label)

    return image, valid_boxes, valid_labels

# draw boxes around detected objects
def draw_boxes(image, boxes, labels):
    draw = ImageDraw.Draw(image)
    try:
        font = ImageFont.truetype("arial.ttf", 20)  
    except IOError:
        font = ImageFont.load_default()  

    for box, label in zip(boxes, labels):
        calorie_info = calories.get(label, "Unknown")
        text = f"{label} ({calorie_info} calories)"
        draw.rectangle([(box[0], box[1]), (box[2], box[3])], outline="red", width=3)
        draw.text((box[0], box[1] - 10), text, fill="blue", font=font)
    return image

image, detected_boxes, detected_labels = detect_objects(new_image_path, detection_model)
image_with_boxes = draw_boxes(image, detected_boxes, detected_labels)
image_with_boxes.show()

c:\Users\tw93\OneDrive\Desktop\masters_project\env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tw93\OneDrive\Desktop\masters_project\env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
Predicted class index: 9, Predicted label: fries
